In [2]:
import numpy as np
from scipy.spatial import distance 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import sklearn
import time

In [3]:
# Load Spotify API key information into env variables. You will need to add in spotify api information
os.environ['SPOTIPY_CLIENT_ID']=''  # "SPOTIPY" is not a typo
os.environ['SPOTIPY_CLIENT_SECRET']=''
os.environ['SPOTIPY_REDIRECT_URI']=''

In [3]:
# Create function which converts a playlist into its mean without touching the categorical variables
def playlist_mean(df):   
    df_avg=pd.DataFrame()
    df_avg.at[0,'danceability']=df['danceability'].mean()
    df_avg.at[0,'energy']=df['energy'].mean()
    df_avg.at[0,'loudness']=df['loudness'].mean()
    df_avg.at[0,'speechiness']=df['speechiness'].mean()
    df_avg.at[0,'acousticness']=df['acousticness'].mean()
    df_avg.at[0,'instrumentalness']=df['instrumentalness'].mean()
    df_avg.at[0,'liveness']=df['liveness'].mean()
    df_avg.at[0,'valence']=df['valence'].mean()
    df_avg.at[0,'tempo']=df['tempo'].mean()
    df_avg.at[0,'duration_ms']=df['duration_ms'].mean()
    df_avg.at[0,'key']=df['key'].mode().iloc[0]
    df_avg.at[0,'mode']=df['mode'].mode().iloc[0]
    df_avg.at[0,'time_signature']=df['time_signature'].mode().iloc[0]


    df_avg=df_avg.astype({"key":'int',"mode":'int',"time_signature":'int'})     # Does not remove the decimal ".0" even if it is an integer!
    df_avg['key']=df_avg['key'].astype(str)     # adding this line seems to convert "key", "mode", and "time_signature" to objects...


    df_avg=df_avg.assign(key_none=0,key_0=0,key_1=0,key_2=0,key_3=0,key_4=0,key_5=0,key_6=0,key_7=0,key_8=0,key_9=0,key_10=0,key_11=0,\
                mode_minor=0,mode_major=0,\
                time_signature_0=0,time_signature_1=0,time_signature_2=0,time_signature_3=0,time_signature_4=0,time_signature_5=0,time_signature_6=0,time_signature_7=0,)


    if df_avg.iloc[0]['key']==-1:
        df_avg.at[0,'key_none']=1
    else:
        col_name='key_'+str(df_avg.iloc[0]['key']) 
        df_avg.at[0,col_name]=1

    col_name='time_signature_'+str(df_avg.iloc[0]['time_signature']) 
    df_avg.at[0,col_name]=1

    if df_avg.iloc[0]['mode']==0:
        df_avg.at[0,'mode_minor']=1
    else:
        df_avg.at[0,'mode_major']=1   

    return df_avg

In [4]:
# Create function which converts a playlist into its weighted average via stddev without touching the categorical variables
def playlist_mean_std(df):   
    df_avg=pd.DataFrame()
    df_avg.at[0,'danceability']=df['danceability'].mean()
    df_avg.at[0,'danceability_std']=df['danceability'].std()
    df_avg.at[0,'energy']=df['energy'].mean()
    df_avg.at[0,'energy_std']=df['energy'].std()
    df_avg.at[0,'loudness']=df['loudness'].mean()
    df_avg.at[0,'loudness_std']=df['loudness'].std()
    df_avg.at[0,'speechiness']=df['speechiness'].mean()
    df_avg.at[0,'speechiness_std']=df['speechiness'].std()
    df_avg.at[0,'acousticness']=df['acousticness'].mean()
    df_avg.at[0,'acousticness_std']=df['acousticness'].std()
    df_avg.at[0,'instrumentalness']=df['instrumentalness'].mean()
    df_avg.at[0,'instrumentalness_std']=df['instrumentalness'].std()
    df_avg.at[0,'liveness']=df['liveness'].mean()
    df_avg.at[0,'liveness_std']=df['liveness'].std()
    df_avg.at[0,'valence']=df['valence'].mean()
    df_avg.at[0,'valence_std']=df['valence'].std()
    df_avg.at[0,'tempo']=df['tempo'].mean()
    df_avg.at[0,'tempo_std']=df['tempo'].std()
    df_avg.at[0,'duration_ms']=df['duration_ms'].mean()
    df_avg.at[0,'duration_ms_std']=df['duration_ms'].std()
    df_avg.at[0,'key']=df['key'].mode().iloc[0]
    df_avg.at[0,'mode']=df['mode'].mode().iloc[0]
    df_avg.at[0,'time_signature']=df['time_signature'].mode().iloc[0]


    df_avg=df_avg.astype({"key":'int',"mode":'int',"time_signature":'int'})     # Does not remove the decimal ".0" even if it is an integer!
    df_avg['key']=df_avg['key'].astype(str)     # adding this line seems to convert "key", "mode", and "time_signature" to objects...


    df_avg=df_avg.assign(key_none=0,key_0=0,key_1=0,key_2=0,key_3=0,key_4=0,key_5=0,key_6=0,key_7=0,key_8=0,key_9=0,key_10=0,key_11=0,\
                mode_minor=0,mode_major=0,\
                     time_signature_0=0,time_signature_1=0,time_signature_2=0,time_signature_3=0,time_signature_4=0,time_signature_5=0,time_signature_6=0,time_signature_7=0)



    if df_avg.iloc[0]['key']==-1:
        df_avg.at[0,'key_none']=1
    else:
        col_name='key_'+str(df_avg.iloc[0]['key']) 
        # df_avg.at[0,col_name]=df_avg.iloc[0]['key']
        df_avg.at[0,col_name]=1

    col_name='time_signature_'+str(df_avg.iloc[0]['time_signature']) 
    # df_avg.at[0,col_name]=df_avg.iloc[0]['time_signature']
    df_avg.at[0,col_name]=1

    if df_avg.iloc[0]['mode']==0:
        df_avg.at[0,'mode_minor']=1
    else:
        df_avg.at[0,'mode_major']=1   

    return df_avg

In [5]:
# Function used to gather playlist information from Spotify
def gather_playlist_data(playlist_uri):
    user_playist_track_uri = []
    for i in range (0,1000,100):
        playlist_info = spotify.playlist_items(playlist_uri, offset=i, limit=100)
        for x in range(0,len(playlist_info['items'])):
            user_playist_track_uri.append(playlist_info['items'][x]['track']['uri'])
        if len(playlist_info['items']) < 100:
            break
    
    return user_playist_track_uri

In [6]:
# Function used to create dummy variables for input to ML model from base track feature data
def dummy_variables(data):
    key_to_add = ['key_none', 'key_0', 'key_1', 'key_2',
    'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10',
    'key_11']
    mode_to_add = ['mode_minor', 'mode_major']
    signature_to_add = ['time_signature_0', 'time_signature_1', 'time_signature_2', 'time_signature_3', 
                        'time_signature_4', 'time_signature_5', 'time_signature_6', 'time_signature_7']
    y = -1
    for x in key_to_add:
        for i in data:
            if i['key'] == y:
                i[x] = 1
            else:
                i[x] = 0
        y+=1
    for i in data:
        if i['mode'] == 1:
            i[mode_to_add[0]] = 0
            i[mode_to_add[1]] = 1
        else:
            i[mode_to_add[0]] = 1
            i[mode_to_add[1]] = 0
    time_signature = 0
    for x in signature_to_add:
        for i in data:
            if i['time_signature'] == time_signature:
                i[x] = 1
            else:
                i[x] = 0
        time_signature +=1

In [7]:
# Function used to gather track feature data
def gather_track_features(uri_track_list):
    b = len(uri_track_list)
    results_full = []
    for i in range(0,b,100):
        if (b - i) < 100:
            x=uri_track_list[i:i+(b-i)]
            y=spotify.audio_features(x)
            results_full = results_full + y
        else:
            x=uri_track_list[i:i+100]
            y=spotify.audio_features(x)
            results_full = results_full + y
        time.sleep(0.5)
    
    return results_full

In [8]:
# Read the playlist_ID_trackURI csv file into a pandas DataFrame
playlist_ID_trackuri = pd.read_csv("./Resources/playlist_ID_trackuri.csv")

# Display DF to confirm data load
playlist_ID_trackuri.head(5)

,pid,track_uri
0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,0,spotify:track:0uqPG793dkDDN7sCUJJIVC
2,0,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,0,spotify:track:0XUfyU2QviPAs6bxSpXYG4
4,0,spotify:track:12qZHAeOyTf93YAWvGDTat


In [9]:
# Read the Playlist clustering csv file into a pandas DataFrame
playlist_weight_df = pd.read_csv("./Resources/scaled_playlist_clusters17.csv")

# Review the DataFrame
playlist_weight_df.head()

,danceability,danceability_std,energy,energy_std,loudness,loudness_std,speechiness,speechiness_std,acousticness,acousticness_std,...,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7,playlist_clusters,pid
0,0.052642,1.068008,0.213927,-0.700676,-0.534153,-0.389743,-0.651185,-0.789438,-0.622047,-0.226146,...,-0.005099,-0.003162,0.0,-0.066555,0.06697,-0.004359,0.0,0.0,1,5121
1,0.579708,0.056448,-0.121494,-0.293096,0.367402,-0.516287,0.328348,1.091248,-0.398160,0.191659,...,-0.005099,-0.003162,0.0,-0.066555,0.06697,-0.004359,0.0,0.0,6,5122
2,0.465851,2.084598,0.185405,-0.055414,0.859986,-1.099622,0.472775,1.303022,-0.067641,0.232899,...,-0.005099,-0.003162,0.0,-0.066555,0.06697,-0.004359,0.0,0.0,4,5123
3,-1.521029,-0.268359,0.883228,0.765240,0.447559,-0.052115,-0.727574,-0.643949,-0.791429,-0.031478,...,-0.005099,-0.003162,0.0,-0.066555,0.06697,-0.004359,0.0,0.0,2,5124
4,0.883569,-0.402855,0.415606,-0.336635,0.493938,-0.719627,1.716434,1.725020,-0.637704,-0.246080,...,-0.005099,-0.003162,0.0,-0.066555,0.06697,-0.004359,0.0,0.0,3,5125


In [10]:
# Load model to test
model_pkl_file = "./Resources/playlist_std_mode_cluster17_model.pkl" 
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)

C:\Users\Joey\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 1.4.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
# Define playlist URI to test
playlist_uri = '4sbjK54KiZ4GzgZ4B2qJWh'

In [12]:
# Establish a connection with the Spotify API
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [13]:
# Gather all track URI's from the user's playlist
user_playist_track_uri = gather_playlist_data(playlist_uri)

# Request track data from spotify on the 5 track slice from the user playlist
data = gather_track_features(user_playist_track_uri)

# Create dummy variable cells for later input into the model
dummy_variables(data)

In [14]:
# Create dataframe from data and create a copy for later use
user_playlist_data_df = pd.DataFrame(data, columns=['danceability','energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
            'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms','key','mode','time_signature', 
            'mode_minor', 'mode_major','key_none', 'key_0', 'key_1', 
            'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 
            'key_9', 'key_10', 'key_11', 'time_signature_0', 'time_signature_1',
            'time_signature_2', 'time_signature_3', 'time_signature_4', 'time_signature_5',
            'time_signature_6', 'time_signature_7'])
user_playlist_model_data_df = user_playlist_data_df.copy()

# Review created dataframe from input playlist data
user_playlist_model_data_df.head(5)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,...,key_10,key_11,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7
0,0.635,0.681,-8.729,0.1120,0.011100,0.816,0.104,0.2350,145.048,audio_features,...,0,0,0,0,0,0,1,0,0,0
1,0.595,0.960,-7.336,0.0675,0.001330,0.926,0.130,0.0999,144.023,audio_features,...,1,0,0,0,0,1,0,0,0,0
2,0.674,0.669,-8.747,0.0518,0.033200,0.716,0.101,0.0834,138.996,audio_features,...,0,0,0,0,0,0,1,0,0,0
3,0.457,0.995,-7.801,0.0454,0.000051,0.865,0.354,0.6850,154.002,audio_features,...,0,0,0,0,0,0,1,0,0,0
4,0.731,0.873,-7.746,0.0518,0.000268,0.851,0.100,0.6710,159.984,audio_features,...,0,1,0,0,0,0,1,0,0,0


In [15]:
# Create the weighted data from the users_playlist
user_playlist_weighted_df = playlist_mean_std(user_playlist_model_data_df)
user_playlist_weighted_df

,danceability,danceability_std,energy,energy_std,loudness,loudness_std,speechiness,speechiness_std,acousticness,acousticness_std,...,mode_minor,mode_major,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7
0,0.626575,0.086274,0.94339,0.072576,-6.434671,2.067756,0.108983,0.097915,0.024681,0.062614,...,0,1,0,0,0,0,1,0,0,0


In [16]:
# Drop columns not needed for modeling
user_playlist_weighted_df = user_playlist_weighted_df.drop(['key', 'mode', 'time_signature'], axis=1)

In [17]:
# Import the scaler used during creation of the playlist ML model
scaler = pickle.load(open('./Resources/scaler_playlist_std_mode_cluster17.sav', 'rb'))

C:\Users\Joey\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
# Scale input_playlist
scaled_user_playlist_data = scaler.transform(user_playlist_weighted_df)
scaled_user_playlist_weighted_df = pd.DataFrame(scaled_user_playlist_data,columns=user_playlist_weighted_df.columns, 
                                            index=user_playlist_weighted_df.index)

In [19]:
# Make cluster prediction using the loaded model
prediction = model.predict(scaled_user_playlist_weighted_df)

# Isolate playlists within predicted cluster
recommend_playlists = playlist_weight_df[playlist_weight_df["playlist_clusters"]==prediction[0]]

# Use cosine similarity to find the 5 closest playlists to the user's playlist within the model
recommend_playlists_df = recommend_playlists.drop(['playlist_clusters'],axis=1)
recommend_playlists_df=recommend_playlists_df.set_index('pid')
recommend_playlists_df['similarity'] = cosine_similarity(recommend_playlists_df.values, scaled_user_playlist_weighted_df.values)
recommend_playlists_df_top_5 = recommend_playlists_df.sort_values('similarity', ascending=False).head(5)

In [20]:
# Review cluster prediction
prediction

array([10])

In [21]:
# Reset index to return the PID data to a column
recommend_playlists_df_top_5 = recommend_playlists_df_top_5.reset_index()

In [22]:
# Gather track uri's from the 5 closest playlists, drop all other columns and remove duplicates
recommended_df = pd.merge(recommend_playlists_df_top_5, playlist_ID_trackuri, on='pid')
recommended_df = recommended_df[['pid','track_uri']]
recommended_df = recommended_df.drop_duplicates(subset=['track_uri'])
recommended_track_uri_df = recommended_df[['track_uri']]

In [23]:
# Compare tracks in the 3 closest playlists to the users playlist to remove those already included
recommended_track_uri_df = recommended_track_uri_df[~recommended_track_uri_df['track_uri'].isin(user_playist_track_uri)]
recommended_track_list = list(recommended_track_uri_df['track_uri'])

# If list empty, return message
if len(recommended_track_list) == 0:
    print("The recommended playlist's contain no songs that were not already in the user's playlist")

In [24]:
# Gather track data from API for the recommended tracks
recommended_track_data = gather_track_features(recommended_track_list)

In [25]:
# Review how many tracks are in the recommendation list
len(recommended_track_list)

130

In [26]:
# Create copy of original user input dataset, to take the mean instead of the weight
user_playlist_mean_data_df = user_playlist_data_df.copy()
user_playlist_mean_data_df = playlist_mean(user_playlist_mean_data_df)
user_playlist_mean_data_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,mode_minor,mode_major,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7
0,0.626575,0.94339,-6.434671,0.108983,0.024681,0.674636,0.207939,0.292864,150.961491,304398.614035,...,0,1,0,0,0,0,1,0,0,0


In [27]:
# Create dummy variable cells for later input into the model
dummy_variables(recommended_track_data)

In [28]:
# Format dataframes for input to model
recommendation_track_data_df = pd.DataFrame(recommended_track_data, columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
            'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
            'time_signature', 'key_none', 'key_0', 'key_1', 
            'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 
            'key_9', 'key_10', 'key_11', 'mode_minor', 'mode_major','time_signature_0', 'time_signature_1',
            'time_signature_2', 'time_signature_3', 'time_signature_4', 'time_signature_5',
            'time_signature_6', 'time_signature_7'])
recommendation_track_data_df=recommendation_track_data_df.set_index('uri')
recommendation_track_data_df = recommendation_track_data_df.drop(['key','type', 'id', 'track_href', 'analysis_url','mode','time_signature'], axis=1)
user_playlist_mean_data_df = user_playlist_mean_data_df.drop(['key', 'mode', 'time_signature'], axis=1)

In [29]:
# Import scaler from track vectors, as this data is different from playlist data above
scaler_vectors = pickle.load(open('./Resources/scaler_track_vectors.sav', 'rb'))

C:\Users\Joey\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [30]:
# Scale recommendation_track_data_df
scaled_recommended_df = scaler_vectors.transform(recommendation_track_data_df)
recommendation_track_data_df = pd.DataFrame(scaled_recommended_df,columns=recommendation_track_data_df.columns, 
                                            index=recommendation_track_data_df.index)


In [31]:
# Scale user_playlist_mean_data_df
scaled_user_df = scaler_vectors.transform(user_playlist_mean_data_df)
user_playlist_mean_data_df = pd.DataFrame(scaled_user_df,columns=user_playlist_mean_data_df.columns, 
                                            index=user_playlist_mean_data_df.index)


In [32]:
# Use cosine similarity to find the closest tracks to the input mean playlist
recommendation_track_data_df['similarity'] = cosine_similarity(recommendation_track_data_df.values, user_playlist_mean_data_df.values)
recommend_tracks_df_top_5 = recommendation_track_data_df.sort_values('similarity', ascending=False)
recommend_tracks_df_top_5.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,mode_major,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7,similarity
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:2JO9pQQKe6AhHxAHwiqIqW,0.187405,0.116946,0.180131,-0.392656,-0.970438,-0.631996,0.046776,1.130841,2.280841,0.198545,...,0.725791,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.683644
spotify:track:4JYDtK3aWUgp80WeIhKVbJ,0.106118,1.218360,0.707077,-0.403075,0.912213,-0.387874,-0.768176,1.723488,-0.532239,-0.895935,...,0.725791,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.485208
spotify:track:6hgBJPTHoDJyxydmtFWHKf,1.471739,0.774787,0.851388,-0.415230,-0.997865,1.145298,-0.802944,-0.984165,0.336489,1.286566,...,0.725791,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.329969
spotify:track:6CjWJYeI8n6634hhQgvscE,0.181986,-1.300233,-1.133057,7.221596,-0.388599,-0.632003,-0.543234,-1.328641,3.100636,0.105497,...,-1.377808,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.251481
spotify:track:0om6UMmvtYsAXYwW6AkoMe,-0.901841,-0.476991,-1.829550,-0.107013,1.394727,1.096644,-0.100727,-0.335958,0.158873,-1.294135,...,-1.377808,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.233239


In [33]:
# Reset index to return the track URI data to a column
recommend_tracks_df_top_5 = recommend_tracks_df_top_5.reset_index()

# Review dataframe
recommend_tracks_df_top_5.head()

,uri,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,mode_major,time_signature_0,time_signature_1,time_signature_2,time_signature_3,time_signature_4,time_signature_5,time_signature_6,time_signature_7,similarity
0,spotify:track:2JO9pQQKe6AhHxAHwiqIqW,0.187405,0.116946,0.180131,-0.392656,-0.970438,-0.631996,0.046776,1.130841,2.280841,...,0.725791,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.683644
1,spotify:track:4JYDtK3aWUgp80WeIhKVbJ,0.106118,1.218360,0.707077,-0.403075,0.912213,-0.387874,-0.768176,1.723488,-0.532239,...,0.725791,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.485208
2,spotify:track:6hgBJPTHoDJyxydmtFWHKf,1.471739,0.774787,0.851388,-0.415230,-0.997865,1.145298,-0.802944,-0.984165,0.336489,...,0.725791,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.329969
3,spotify:track:6CjWJYeI8n6634hhQgvscE,0.181986,-1.300233,-1.133057,7.221596,-0.388599,-0.632003,-0.543234,-1.328641,3.100636,...,-1.377808,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.251481
4,spotify:track:0om6UMmvtYsAXYwW6AkoMe,-0.901841,-0.476991,-1.829550,-0.107013,1.394727,1.096644,-0.100727,-0.335958,0.158873,...,-1.377808,-0.03769,-0.103107,0.0,-0.335807,0.389518,-0.137138,0.0,0.0,0.233239


In [34]:
# Gather 5 track URI's from the recommended dataframe
recommended_track_uri_list = []
for x in range(0,5,1):
    recommended_track_uri_list.append(recommend_tracks_df_top_5['uri'][x])

# Gather information from spotify on the 5 songs to recommend
tracks_info = spotify.tracks(recommended_track_uri_list)

In [35]:
# Compile recommended track data
results = []
for x in range(0,len(recommended_track_uri_list)):
    result_dict = {}
    result_dict['Track URI'] = tracks_info['tracks'][x]['uri']
    result_dict['Album Cover'] = tracks_info['tracks'][x]['album']['images'][0]['url']
    result_dict['Track Name'] = tracks_info['tracks'][x]['name']
    result_dict['Artist Name'] = tracks_info['tracks'][x]['artists'][0]['name']
    result_dict['Preview URL'] = tracks_info['tracks'][x]['preview_url']
    results.append(result_dict)

In [36]:
# Review results dictionary
results

[{'Track URI': 'spotify:track:2JO9pQQKe6AhHxAHwiqIqW',
  'Album Cover': 'https://i.scdn.co/image/ab67616d0000b2735553e46956682d76352471ea',
  'Track Name': 'Candy Rain (Re-Recorded / Remastered)',
  'Artist Name': 'Soul For Real',
  'Preview URL': 'https://p.scdn.co/mp3-preview/a18c11a16fedb126d3137ead10cdf5d0e6045d20?cid=eec42f936e1b4cfcab34e7980d9fe51e'},
 {'Track URI': 'spotify:track:4JYDtK3aWUgp80WeIhKVbJ',
  'Album Cover': 'https://i.scdn.co/image/ab67616d0000b273247507a839aab9929358572a',
  'Track Name': 'Nébuleux Bonhomme',
  'Artist Name': 'Françoiz Breut',
  'Preview URL': 'https://p.scdn.co/mp3-preview/fdbfbbc483fb882635eadf0abe4c3d54797d0a26?cid=eec42f936e1b4cfcab34e7980d9fe51e'},
 {'Track URI': 'spotify:track:6hgBJPTHoDJyxydmtFWHKf',
  'Album Cover': 'https://i.scdn.co/image/ab67616d0000b2738ac778cc7d88779f74d33311',
  'Track Name': "Rollin' & Scratchin'",
  'Artist Name': 'Daft Punk',
  'Preview URL': 'https://p.scdn.co/mp3-preview/bb165c4d5cb7b89a7dec1dccc2d38571a528c9db?